In [1]:
# https://towardsdatascience.com/how-to-use-the-staple-algorithm-to-combine-multiple-image-segmentations-ce91ebeb451e
# packages
import nibabel as nib # https://nipy.org/nibabel/
import SimpleITK as sitk # https://simpleitk.org/
from matplotlib import pyplot as plt
import os
from meidic_vtach_utils.run_on_recommended_cuda import get_cuda_environ_vars as get_vars
os.environ.update(get_vars(select="*"))
from collections import OrderedDict
import torch
import numpy as np


### Recommended gpus on this machine (descending order) ###
  ID  Card name                     Util    Mem free  Cuda             User(s)
----  --------------------------  ------  ----------  ---------------  -------------------
   1  NVIDIA GeForce RTX 2080 Ti     0 %   11016 MiB  11.5(495.29.05)
   2  NVIDIA GeForce RTX 2080 Ti     0 %    9937 MiB  11.5(495.29.05)  grossbroehmer
   0  NVIDIA GeForce RTX 2080 Ti     0 %    1264 MiB  11.5(495.29.05)  ehrhardt, schneider
   3  NVIDIA GeForce RTX 2080 Ti  ! 61 %    1485 MiB  11.5(495.29.05)  ehrhardt, nicke

Will apply following mapping

  ID  Card name                       torch
----  --------------------------  --  -------
   1  NVIDIA GeForce RTX 2080 Ti  ->  cuda:0


In [2]:
DATA_PATH = "/share/data_supergrover1/weihsbach/shared_data/important_data_artifacts/curriculum_deeplab/20220114_crossmoda_multiple_registrations/crossmoda_deeds_registered.pth"
bare_data = torch.load(DATA_PATH)

In [8]:
reg_state = "acummulate_deeds_FT2_MT1"

staple_filter = sitk.STAPLEImageFilter()
# sitk.ProcessObject.SetGlobalDefaultDebugOff()
FOREGROUND = 1.0
weight_data = {}
EVERY = 1
staple_filter.SetForegroundValue(FOREGROUND)
DEBUG = False
if reg_state == "acummulate_deeds_FT2_MT1":
    for fixed_id, moving_dict in bare_data.items():
        # if fixed_id != '111l': continue
        print(fixed_id)
        # print(moving_dict)
        sorted_moving_dict = OrderedDict(moving_dict)
        for slice_idx in range(45,95):
            # slice_idx = 64
            print(slice_idx)
            moving_data = []
            selected_moving_ids = []
            # staple_filter = sitk.STAPLEImageFilter()
            # staple_filter.SetForegroundValue(FOREGROUND)

            for idx_mov, (moving_id, moving_sample) in enumerate(sorted_moving_dict.items()):
                # Only use every third warped sample
                print(idx_mov)
                # idx_mov = 29
                if idx_mov % EVERY == 0:
                    moving_data.append(moving_sample['warped_label'][slice_idx].cpu())
                    moving_slice_id = f"{fixed_id}:m{moving_id}W{slice_idx-45:03d}"
                    selected_moving_ids.append(moving_slice_id)

            sitk_moving_data = [sitk.GetImageFromArray(reg_seg.to_dense().numpy().astype(np.int16)) for reg_seg in moving_data]
            _ = staple_filter.Execute(sitk_moving_data)
            # del staple_filter
            # del sitk_moving_data
            # staple_out = sitk.STAPLE(sitk_moving_data, FOREGROUND)
            # consensus = sitk.GetArrayFromImage(staple_out)
            
            # specitivity = staple_filter.GetSpecificity()
            # sensitivity = staple_filter.GetSensitivity()
            # specitivity = sensitivity = [0]*60
            # f_weight_dict = weight_data.get(fixed_id, {})
            # staple_consensus = sitk.GetArrayFromImage(staple_out)
            # for moving_id, sens, spec in zip(selected_moving_ids, sensitivity, specitivity):
            #     weight_data[moving_id] = dict(sensitivity=sens, specitivity=spec)
            # weight_data[fixed_id] = f_weight_dict
        
        if DEBUG: break
else:
    raise ValueError()

# weight_data['data_path'] = DATA_PATH
# torch.save(weight_data, f"./data/staple_calc/{reg_state}_every_{EVERY}_2dslices.pth")


108r
45
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
46
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
47
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
48
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
49
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
50
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
51
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
52
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
53
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
54
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
55
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
56
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29

In [ ]:
dps = []
for key in weight_data.keys():
    if key == 'data_path': continue
    if np.isnan(weight_data[key]['sensitivity']):
        pass
    else:
        dps.append(weight_data[key]['sensitivity'])
plt.hist(dps)

In [ ]:
network_scores = torch.load("/share/data_supergrover1/weihsbach/shared_data/tmp/curriculum_deeplab/data/output/noble-forest-1144_fold0_epx39/train_label_snapshot.pth")

In [ ]:
d_ids = network_scores['d_ids']
network_parameter = network_scores['data_parameters']

In [ ]:
set(d_ids) == set(weight_data.keys())

In [ ]:
print(sorted(d_ids)[:10])
print(sorted(weight_data.keys())[:10])
bare_data.keys()



In [ ]:
print(list(weight_data.keys()))